In [1]:
import os
import sys
from glob import glob
import yaml
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import scienceplots
plt.style.use('science')
base_dir = os.path.join(os.getcwd(), '..')
sys.path.append(base_dir)

In [2]:
with open('../config/svgp.yaml', 'r') as f:
    cfg = yaml.safe_load(f)
    cfg['dataset']['path'] = os.path.join('../', cfg['dataset']['path'])

### Estimate stddev of haversine distance

In [5]:
import run_svgp_vertical_profile as main
from src.kernels.haversine_matern import batch_haversine_distances

data = main.make_data(cfg)

torch.random.manual_seed(5)
rdm_idx = torch.randperm(len(data.x))[:4000]
latlon_subset = data.x[None, rdm_idx, 2:4]
hav = batch_haversine_distances(latlon_subset, latlon_subset).squeeze()
hav = hav[np.triu_indices(4000)]
hav_stddev = torch.std(hav)

/home/bouabid/.pyenv/versions/sandbox/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


## SVGP

In [6]:
import run_svgp_vertical_profile as main

data = main.make_data(cfg)
model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = model.kernel.kernels[0].kernels[0].outputscale.detach().item()
    ltime = model.kernel.kernels[0].kernels[0].base_kernel.lengthscale[0][0].detach().item()
    llatlon = model.kernel.kernels[0].kernels[1].lengthscale[0][0].detach().item() / hav_stddev
    lmeteo = model.kernel.kernels[1].lengthscale[0].detach()
    lP, lT, lRH, lomega = lmeteo
    lP, lT, lRH, lomega = lP.item(), lT.item(), lRH.item(), lomega.item()
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_T$', r'$\ell_P$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lT, lP, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [7]:
svgp_paths = glob('../experiments/data/outputs/svgp/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df = params_df.aggregate(['mean', 'std'])
# params_df.to_latex('./tables/svgp-hyperparameters.tex', escape=False)
params_df

/tmp/ipykernel_70798/1449306794.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  params_df.to_latex('./tables/svgp-hyperparameters.tex', escape=False)


,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_T$,$\ell_P$,$\ell_{RH}$,$\ell_\omega$
mean,0.263053,7.606869,0.852344,2.508282,3.377943,3.382885,6.574384
std,0.017924,0.194535,0.058697,0.063753,0.097788,0.092575,0.088053


## GP only

In [8]:
import ablations.run_svgp_gp_only as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = model.kernel.kernels[0].kernels[0].outputscale.detach().item()
    ltime = model.kernel.kernels[0].kernels[0].base_kernel.lengthscale[0][0].detach().item()
    llatlon = model.kernel.kernels[0].kernels[1].lengthscale[0][0].detach().item() / hav_stddev
    lmeteo = model.kernel.kernels[1].lengthscale[0].detach()
    lP, lT, lRH, lomega = lmeteo
    lP, lT, lRH, lomega = lP.item(), lT.item(), lRH.item(), lomega.item()
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_T$', r'$\ell_P$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lT, lP, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [9]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-gp-only/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df = params_df.aggregate(['mean', 'std'])
# params_df.to_latex('./tables/svgp-gp-only-hyperparameters.tex', escape=False)
params_df

/tmp/ipykernel_70798/3543131209.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  params_df.to_latex('./tables/svgp-gp-only-hyperparameters.tex', escape=False)


,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_T$,$\ell_P$,$\ell_{RH}$,$\ell_\omega$
mean,0.268129,6.612070,0.887976,2.614172,3.459010,3.372861,7.877330
std,0.017978,0.113539,0.040572,0.082568,0.067346,0.086094,0.290475


## Spatiotemporal only

In [10]:
import ablations.run_svgp_spatiotemporal_only as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = np.nan
    ltime = model.kernel.kernels[0].lengthscale[0][0].detach().item()
    llatlon = model.kernel.kernels[1].lengthscale[0][0].detach().item() / hav_stddev
    lT, lP, lRH, lomega = np.nan, np.nan, np.nan, np.nan
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_T$', r'$\ell_P$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lT, lP, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [11]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-spatiotemporal-only/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df = params_df.aggregate(['mean', 'std']).fillna('-')
# params_df.to_latex('./tables/svgp-spatiotemporal-only-hyperparameters.tex', escape=False)
params_df

/tmp/ipykernel_70798/111661997.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  params_df.to_latex('./tables/svgp-spatiotemporal-only-hyperparameters.tex', escape=False)


,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_T$,$\ell_P$,$\ell_{RH}$,$\ell_\omega$
mean,-,6.715111,1.077420,-,-,-,-
std,-,0.337456,0.019803,-,-,-,-


## Meteorological only

In [12]:
import ablations.run_svgp_meteorological_only as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = np.nan
    ltime = np.nan
    llatlon = np.nan
    lP, lT, lRH, lomega = model.kernel.lengthscale[0].detach()
    lP, lT, lRH, lomega = lP.item(), lT.item(), lRH.item(), lomega.item()
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_T$', r'$\ell_P$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lT, lP, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [13]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-meteorological-only/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df = params_df.aggregate(['mean', 'std']).fillna('-')
# params_df.to_latex('./tables/svgp-meteorological-only-hyperparameters.tex', escape=False)
params_df

/tmp/ipykernel_70798/420966973.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  params_df.to_latex('./tables/svgp-meteorological-only-hyperparameters.tex', escape=False)


,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_T$,$\ell_P$,$\ell_{RH}$,$\ell_\omega$
mean,-,-,-,1.644091,3.002588,2.525721,6.807148
std,-,-,-,0.035216,0.065182,0.058375,0.117395


## Product kernel

In [14]:
import ablations.run_svgp_product_kernel as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gamma = model.kernel.kernels[0].outputscale.detach().item()
    ltime = model.kernel.kernels[0].base_kernel.lengthscale[0][0].detach().item()
    llatlon = model.kernel.kernels[1].lengthscale[0][0].detach().item() / hav_stddev
    lP = model.kernel.kernels[2].lengthscale[0][0].detach().item()
    lT = model.kernel.kernels[3].lengthscale[0][0].detach().item()
    lRH = model.kernel.kernels[4].lengthscale[0][0].detach().item()
    lomega = model.kernel.kernels[5].lengthscale[0][0].detach().item()
    columns = [r'$\gamma_1$', r'$\ell_t$', r'$\ell_{latlon}$',
               r'$\ell_T$', r'$\ell_P$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gamma, ltime, llatlon, lT, lP, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [15]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-product-kernel/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = pd.concat(dfs, axis=0).reset_index(drop=True)
params_df = params_df.aggregate(['mean', 'std'])
# params_df.to_latex('./tables/svgp-product-kernel-hyperparameters.tex', escape=False)
params_df

/tmp/ipykernel_70798/93995452.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  params_df.to_latex('./tables/svgp-product-kernel-hyperparameters.tex', escape=False)


,$\gamma_1$,$\ell_t$,$\ell_{latlon}$,$\ell_T$,$\ell_P$,$\ell_{RH}$,$\ell_\omega$
mean,0.919986,8.795509,1.205547,5.597897,6.177846,5.792094,8.324606
std,0.014222,0.072981,0.000567,0.133629,0.128613,0.089685,0.098687


## Additive kernel

In [16]:
import ablations.run_svgp_additive_kernel as main

model = main.make_model(cfg, data)

def extract_params_svgp(state_dict_path):
    state_dict = torch.load(state_dict_path, map_location='cpu')
    model.load_state_dict(state_dict)
    gammatime = model.kernel.kernels[0].outputscale.detach().item()
    ltime = model.kernel.kernels[0].base_kernel.lengthscale[0][0].detach().item()
    
    gammalatlon = model.kernel.kernels[1].outputscale.detach().item()
    llatlon = model.kernel.kernels[1].base_kernel.lengthscale[0][0].detach().item() / hav_stddev
    
    gammaP = model.kernel.kernels[2].outputscale.detach().item()
    lP = model.kernel.kernels[2].base_kernel.lengthscale[0][0].detach().item()
    
    gammaT = model.kernel.kernels[3].outputscale.detach().item()
    lT = model.kernel.kernels[3].base_kernel.lengthscale[0][0].detach().item()

    gammaRH = model.kernel.kernels[4].outputscale.detach().item()
    lRH = model.kernel.kernels[4].base_kernel.lengthscale[0][0].detach().item()
    
    gammaomega = model.kernel.kernels[5].outputscale.detach().item()
    lomega = model.kernel.kernels[5].base_kernel.lengthscale[0][0].detach().item()
    
    columns = [r'$\gamma_t$', r'$\gamma_{latlon}$', r'$\gamma_T$', r'$\gamma_P$', r'$\gamma_{RH}$', r'$\gamma_\omega$',
               r'$\ell_t$', r'$\ell_{latlon}$', r'$\ell_T$', r'$\ell_P$', r'$\ell_{RH}$', r'$\ell_\omega$']
    params_vals = [[gammatime, gammalatlon, gammaT, gammaP, gammaRH, gammaomega,
                    ltime, llatlon, lT, lP, lRH, lomega]]
    params_df = pd.DataFrame(data=params_vals, columns=columns)
    return params_df

In [17]:
svgp_paths = glob('../experiments/data/outputs/ablation/svgp-additive-kernel/seed_*')
dfs = []
for path in svgp_paths:
    params_df = extract_params_svgp(os.path.join(path, 'state_dict.pt'))
    dfs.append(params_df)
params_df = params_df.aggregate(['mean', 'std'])
# params_df.to_latex('./tables/svgp-additive-kernel-hyperparameters.tex', escape=False)
params_df

/tmp/ipykernel_70798/511192269.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  params_df.to_latex('./tables/svgp-additive-kernel-hyperparameters.tex', escape=False)


,$\gamma_t$,$\gamma_{latlon}$,$\gamma_T$,$\gamma_P$,$\gamma_{RH}$,$\gamma_\omega$,$\ell_t$,$\ell_{latlon}$,$\ell_T$,$\ell_P$,$\ell_{RH}$,$\ell_\omega$
mean,1.483759,0.252393,0.436381,0.32873,0.206365,0.135793,4.389093,0.604549,3.401802,3.062854,3.311474,4.177647
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
